# Carbon emissions of __remotely__ executed code

As we can't measure the carbon footprint generated by remotely executed code via an API request, the package API Emissions Tracker was developed to estimate it. Check out the [GitHub repository](https://github.com/borisruf/api_emissions_tracker) for more details.

### First steps
Let's start with installing the package: energy optimization

In [ ]:
!pip install git+https://github.com/borisruf/api_emissions_tracker.git

Import the MockAzureOpenAI class to simulate requests to the Azure OpenAI API. Also import the APIEmissionsTracker class:

In [ ]:
from mockai import MockAzureOpenAI
from api_emissions_tracker import APIEmissionsTracker

Specify the model you want to test. No need to worry about the endpoint and credentials as we use a mocked environment.

In [ ]:
AZURE_ENDPOINT = "[YOUR_AZURE_ENDPOINT]"
OPENAI_MODEL = "gpt-35-turbo"   # specify a supported model from the list
OPENAI_API_KEY = "[YOUR_OPENAI_API_KEY]"

Start the tracking:

In [ ]:
tracker = APIEmissionsTracker()
tracker.start()

Make the API request. The syntax of MockAzureOpenAI follows [AzureOpenAI](https://github.com/openai/openai-python?tab=readme-ov-fil#microsoft-azure-openai). For supported prompts, check the [MockAI documentation](https://github.com/borisruf/mockai?tab=readme-ov-file#usage).

In [ ]:
client = MockAzureOpenAI(azure_endpoint=AZURE_ENDPOINT, api_key=OPENAI_API_KEY)
response = client.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "system", "content": "What is the origin of the Olympic Games?"}])

Print the output.

In [ ]:
print(response.choices)

Print the total emissions.

In [ ]:
tracker.stop()

# A quick look at the carbon emissions of a token
We have been working on estimating the carbon emissions of LLMs, in particular SecureGPT.  
The estimates were based on HuggingFace's [LLM Perf-Leaderboard](https://huggingface.co/spaces/optimum/llm-perf-leaderboard), which provides data on the energy consumption required to process open-source model tokens.

Using this method, we estimated that a token sent or generated by a SecureGPT consumes an average of 0.00000762501875 kWh, and therefore emits [2.46 mg CO2e](https://borisruf.github.io/carbon-footprint-modeling-tool/index.html?id=gpt-ruf-mortas-token&emission_type=co2e) if the service is located on the West side of the USA.

One way of reducing LLM energy consumption is to reduce the number of tokens in and out.  
For example, by fine-tuning the model to obtain short, structured responses.

### Example of fine-tuning to reduce the number of output tokens

In [ ]:
data = {
    'text': [
        """
        Summarize the following text:
        [A text on climate change and agriculture]
        """,
        """
        Summarize the following text:
        [A text on exercice and mental health]
        """,
        """
        Summarize the following text:
        [A text on benefits and challenges of remote work]
        """,
        """
        Summarize the following text:
        [A text on air pollution and human health]
        """,
        """
        Summarize the following text:
        [A text on impacts of artificial intelligence on environment]
        """,
    ],
    'labels': [
        """
        # Title : Climate Change Impact on Global Agriculture
        # Introduction : This report discusses the impact of climate change on agriculture worldwide, focusing on crop yields and food security.
        # Keypoints :
        - Rising temperatures and changing precipitation patterns can negatively affect crop yields.
        - Increased frequency of extreme weather events (droughts and floods), poses risks to agricultural systems.
        - Lead to the spread of pests and diseases, threatening crop health.
        # Conclusion : Adaptation strategies, including development of climate-resilient crops and sustainable farming practices, are crucial for ensuring future food security.
        """,
        """
        # Title : Exercise and Mental Health Benefits
        # Introduction : This article explores the positive effects of regular exercise on mental health, focusing on mood enhancement and stress reduction.
        # Key Points :
        - Exercise releases endorphins, which are natural mood lifters and can help reduce feelings of anxiety and depression.
        - Regular physical activity can improve sleep quality and reduce symptoms of insomnia.
        - Exercise can also boost self-esteem and cognitive function, promoting overall mental well-being.
        # Conclusion : Incorporating exercise into daily routines can significantly improve mental health and overall quality of life.
        """,
        """
        ...
        """,
        """
        ...
        """,
        """
        ...
        """
    ]
}

Other methods and best practices can be used to optimize inference.  

We can specify in a question about code, that we only want the code and not the explanation.

With each new message in a conversation with an LLM chatbot, all previous messages are included in order to preserve context. However, this considerably increases the number of tokens in the question.

To avoid this, we can, for example, choose to launch a new conversation if we change the subject and the previous context is no longer important.

When the previous context is important, but the conversation becomes super long, we can ask the chatbot to generate a summary of the context, adjust it, and put it back on top of our first message in a new conversation.

Also, when the answer isn't satisfactory and you want to ask the question again, refining exactly what you're writting, it's advisable not to resend a message in a row with the improved question, but rather to edit the message (when possible) and resend it modified. This avoids keeping a useless history without spamming the conversation history by starting the discussion all over again.

Finally, you can also choose to stop generating the answer when you see that the answer is not at all what you expected.

# Recommendations on technological tools and choice of infrastructure
## Tools and technologies
### Use of energy-efficient tools
Use tools and libraries that are designed to optimize computing resources and GPU (NumPy, TensorFlow, PyTorch, ...).

Follow green coding good practices. For example: https://medium.com/@moustaphadaka

Quantization is a process that reduces the precision of numerical data. In power efficiency, it's used to lower the bit precision in digital systems and machine learning models. This leads to less memory usage, faster computations, and reduced power consumption. However, it can introduce errors, so a balance between efficiency and accuracy is needed.

### Choose the right model
Instead of re-training a model, a similar to the one you want to create may already exist. You can finetune a model close to yours to suit your needs.

Choose the right model for your needs:
- choose a specialized model rather than a general one,
- balance between necessary accuracy, size and energy performance.

For LLMs inference, reduce the number of tokens in the instruction prompt and in the answers

### Tools for quantifying environmental impact
Measure and estimate the environmental impact of projects with adapted libraries. Other tools, such as the Green Algorithms Project, offer methods and best practices for designing more environmentally-friendly algorithms.

*References:* 
- https://codecarbon.io/ (Python)
- https://github.com/lfwa/carbontracker (Python)
- https://developers.thegreenwebfoundation.org/co2js/overview/ (JavaScript)
- https://calculator.green-algorithms.org/ (Server)

## Choice of infrastructure
### Use of green infrastructures
Opt for data centers powered by renewable energies and committed to a good ESG policy.

### Choose low carbon intensity location
- Norway: 0.0133 kg CO2e/kWh
- Sweden: 0.0256 kg CO2e/kWh
- France: 0.0814 kg CO2e/kWh
- Switzerland: 0.0869 kg CO2e/kWh
- Finland: 0.1048 kg CO2e/kWh
- Denmark: 0.1348 kg CO2e/kWh
- Brazil: 0.1426 kg CO2e/kWh
- Canada: 0.1546 kg CO2e/kWh
- Belgium: 0.206 kg CO2e/kWh
- Austria: 0.2061 kg CO2e/kWh
- Spain: 0.2086 kg CO2e/kWh
- Ireland: 0.3476 kg CO2e/kWh

*source: https://www.iea.org/data-and-statistics/data-product/emissions-factors-2022*